This Notebook has the single purpose of creating a single file out of the resource found here: https://labs.acousticbrainz.org/million-song-dataset-echonest-archive/

Since the Million Song Data Set was created with outdated IDs this allows to remap ids to other apis such as Spotify etc.

1. Download file from link ftp://ftp.acousticbrainz.org/pub/acousticbrainz/acousticbrainz-labs/download/msdrosetta/millionsongdataset_echonest.tar.bz2
2. Unpack in '/data/raw/MillionSongFullSummary/mapping/' (~3.5GB)
3. Run this Notebook to create 'mapping.csv' (reading in these files is very resource intensive and is going to create autosave points temporarily, which will be removed once the files are merged)
4. You can now run the notebook -> Million Song Data Set Wrangling

In [9]:
import os
import re
import h5py
import pandas as pd
from glob import glob, iglob
import json

In [10]:
%cd ../data/raw

[Errno 2] No such file or directory: '../data/raw'
/Users/sebastian/git_repos/data_science/hit_predictor/data/raw


In [8]:
output_filepath = '../interim/'
temp_dir = 'mapping_temp'
skip = 0

if not os.path.exists(output_filepath + temp_dir):
    os.mkdir(output_filepath + temp_dir)
else:
    for file in iglob(output_filepath + temp_dir + '/*.csv'):
        m = re.search('mapping_([0-9]+?).csv$', file)
        if m:
            file_nr = int(m.group(1))
            if file_nr > skip:
                skip = file_nr

filenames_all = glob('MillionSongFullSummary/mapping/**/*.json',recursive=True)

def chunks(l, n):
    """Yield successive n-sized chunks from l."""
    for i in range(0, len(l), n):
        yield l[i:i + n]

for i,filenames in enumerate(chunks(filenames_all,10000)):
    mapped_songs_dfs = []
    file_nr = i + 1
    if file_nr <= skip:
        print('skip:', file_nr)
        continue
    print('work on:', file_nr)
    for filename in filenames:
        with open(filename) as data_file:    
            data = json.load(data_file)
        mapped_song = pd.io.json.json_normalize(data=data['response']['songs'], record_path=['tracks'], 
                                      record_prefix='tracks.',
                                      meta=['artist_name','artist_id','id','title'])
        mapped_song = mapped_song.reindex(['tracks.catalog','tracks.foreign_id','tracks.id','artist_name','id','title'],axis='columns')
        mapped_song = mapped_song.loc[(mapped_song['tracks.catalog'] == 'spotify') | (mapped_song['tracks.catalog'] == 'musicbrainz'),:]
        mapped_songs_dfs.append(mapped_song)

    mapping_df = pd.concat(mapped_songs_dfs).reset_index(drop=True)
    mapping_df.to_csv(output_filepath + temp_dir + '/mapping_' + str(file_nr) + '.csv',sep='\t',index=False,encoding='utf-8')

print('done')

KeyboardInterrupt: 

In [ ]:
subfiles_mapping = [pd.read_csv(file,sep='\t') for file in iglob(output_filepath + temp_dir + '/mapping_*.csv')]

mapping_df = pd.concat(subfiles_mapping).reset_index(drop=True)
mapping_df.to_csv(output_filepath + '/mapping_summary.csv',sep='\t',index=False,encoding='utf-8')

In [11]:
# Remove Temp files and directory - Cleaning Up
[os.remove(file) for file in iglob(output_filepath + temp_dir + '/mapping_*.csv')]
try:
    os.remove(output_filepath + temp_dir + '/.DS_Store')
except:
    print('No .DS_Store file. You\'re all set!')
    
os.rmdir(output_filepath + temp_dir)

No .DS_Store file. You're all set!
